# Route and Solve Agent  Agent with LangChain, Granite and watsonx.ai

In this recipe, you will use the IBM [Granite](https://www.ibm.com/granite) model now available on watsonx.ai to build a Route and Solve Agent. This agent extends the [Function Calling Agent](../Function_Calling/Function_Calling_Agent.ipynb) recipe by introducing a router node that intelligently distributes queries to specialized subagents, each with their own grouped set of tools.

The Route and Solve architecture is ideal when you have a large number of tools that can be naturally grouped by category or domain. Instead of presenting all tools to a single agent, the router first determines which category of tools are needed, then routes the query to the appropriate subagent.

This approach offers several benefits:
- **Reduced tool selection errors**: Subagents only see relevant tools for their domain
- **Better scalability**: Easy to add new tool categories as subagents
- **Improved reasoning**: Specialized subagents can reason more effectively within their domain
- **Clear separation of concerns**: Tools are logically grouped by functionality


## Key Concepts in Route and Solve Architecture

### 1. **Router Node with Tool Calling**
The router uses the LLM's native tool-calling capability to determine which subagent should handle the query. Each subagent is represented as an actual callable tool that the router can invoke directly. This is more scalable than traditional routing approaches and leverages the LLM's semantic understanding for intelligent routing decisions.

### 2. **Subagents as Callable Tools**
Instead of just routing to a subagent name, each subagent is a proper tool that:
- Accepts the user query as input
- Executes the subagent's internal logic and tools
- Returns the actual result
- Is invoked directly by the LLM through tool calling

Examples:
- **Finance Agent Tool**: Accepts a query, executes the finance subagent with its tools, returns stock data
- **Weather Agent Tool**: Accepts a query, executes the weather subagent with its tools, returns weather data

### 3. **Subagent Implementation**
Each subagent is a specialized function calling agent that:
- Only has access to tools in its category
- Maintains an internal loop structure using LangGraph
- Determines if tool calls are needed, executes them, and iterates
- Returns the final response to the router

### 4. **Scalability**
Adding new tool categories is straightforward and modular:
1. Define new tools in a new category
2. Create a new subagent graph using `create_agent`
3. Define a new subagent tool function that wraps the subagent
4. Add the tool to the router's tool list
5. The router automatically gains the capability without modifying graph structure

This is fundamentally more extensible than manually adding nodes and edges.

## Agent Flow Diagram

![Agent Flow Example](./images/route_solve_flow.png)

This diagram shows how queries flow through the Route and Solve agent:
1. A user query enters the system at the **Router Node**
2. The **Router Node** uses tool calling to analyze the query and determine which tool/subagent to invoke
3. The appropriate **Subagent Tool** is invoked directly (Finance Agent Tool or Weather Agent Tool)
4. The subagent executes its internal logic, calling domain-specific tools as needed
5. The final response is returned to the user

Key difference from traditional routing: Subagents are actual callable tools that execute directly, not just routing destinations.

# Steps

## Step 1. Set up your environment

While you can choose from several tools, this recipe is best suited for a Jupyter Notebook. Jupyter Notebooks are widely used within data science to combine code with various data sources such as text, images and data visualizations.

You can run this notebook in [Colab](https://colab.research.google.com/), or download it to your system and [run the notebook locally](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Getting_Started_with_Jupyter_Locally/Getting_Started_with_Jupyter_Locally.md).

To avoid Python package dependency conflicts, we recommend setting up a [virtual environment](https://docs.python.org/3/library/venv.html).

Note, this notebook is compatible with Python 3.12 and well as Python 3.11, the default in Colab at the time of publishing this recipe. To check your python version, you can run the `!python --version` command in a code cell.

## Step 2. Set up a watsonx.ai instance

See [Getting Started with IBM watsonx](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Getting_Started/Getting_Started_with_WatsonX.ipynb) for information on getting ready to use watsonx.ai.

You will need three credentials from the watsonx.ai set up to add to your environment: `WATSONX_URL`, `WATSONX_APIKEY`, and `WATSONX_PROJECT_ID`.

## Step 3. Install relevant libraries and set up credentials and the Granite model

We'll need a few libraries for this recipe. We will be using LangGraph and LangChain libraries to use Granite on watsonx.ai.

In [ ]:
! echo "::group::Install Dependencies"
%pip install uv
! uv pip install "git+https://github.com/ibm-granite-community/utils.git" \
    langgraph \
    langchain \
    langchain_ibm
! echo "::endgroup::"

Now we will get the credentials to use watsonx.ai and create the Granite model for use.

In [ ]:
from ibm_granite_community.notebook_utils import get_env_var
from langchain_core.utils.utils import convert_to_secret_str
from langchain.chat_models import init_chat_model

model = "ibm/granite-4-h-small"

llm_params = {
    "temperature": 0,
    "max_completion_tokens": 200,
    "repetition_penalty": 1.05,
}

# --- 1. LLM Initialization (Agent's Brain) ---
llm = init_chat_model(
    model=model,
    model_provider="ibm",
    url=convert_to_secret_str(get_env_var("WATSONX_URL")),
    apikey=convert_to_secret_str(get_env_var("WATSONX_APIKEY")),
    project_id=get_env_var("WATSONX_PROJECT_ID"),
    params=llm_params,
)
print(f"LLM initialized: {model}")



## Step 4: Define the functions grouped by category

In a Route and Solve agent, tools are organized into logical groups. Each group will be handled by a dedicated subagent. This organization allows the router to determine which category of tools is needed and route the query accordingly.

We'll create two categories: **Finance Tools** and **Weather Tools**. In a real-world scenario, you might have more categories such as Database Tools, Email Tools, or API Integration Tools.

These tools are sourced from the [Function Calling Agent](../Function_Calling/Function_Calling_Agent.ipynb) recipe to ensure consistency across recipes and enable maximum code reuse.

### Finance Tools

In [ ]:
import requests
from langchain_core.utils.utils import convert_to_secret_str

AV_STOCK_API_KEY = convert_to_secret_str(get_env_var("AV_STOCK_API_KEY", "unset"))

def get_stock_price(ticker: str, date: str) -> dict:
    """
    Retrieves the lowest and highest stock prices for a given ticker and date.

    Args:
        ticker: The stock ticker symbol, for example, "IBM".
        date: The date in "YYYY-MM-DD" format for which you want to get stock prices.

    Returns:
        A dictionary containing the low and high stock prices on the given date.
    """
    print(f"Getting stock price for {ticker} on {date}")

    apikey = AV_STOCK_API_KEY.get_secret_value()
    if apikey == "unset":
        print("No API key present; using a fixed, predetermined value for demonstration purposes")
        return {
            "low": "245.4500",
            "high": "249.0300"
        }

    try:
        stock_url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&apikey={apikey}"
        stock_data = requests.get(stock_url)
        data = stock_data.json()
        stock_low = data["Time Series (Daily)"][date]["3. low"]
        stock_high = data["Time Series (Daily)"][date]["2. high"]
        return {
            "low": stock_low,
            "high": stock_high
        }
    except Exception as e:
        print(f"Error fetching stock data: {e}")
        return {
            "low": "none",
            "high": "none"
        }

print("=== Finance Tools Initialized ===")
print("  Tool 1: 'get_stock_price'")

### Weather Tools

In [ ]:
WEATHER_API_KEY = convert_to_secret_str(get_env_var("WEATHER_API_KEY", "unset"))

def get_current_weather(location: str) -> dict:
    """
    Fetches the current weather for a given location (default: San Francisco).

    Args:
        location: The name of the city for which to retrieve the weather information.

    Returns:
        A dictionary containing weather information such as temperature in celsius, weather description, and humidity.
    """
    print(f"Getting current weather for {location}")
    apikey=WEATHER_API_KEY.get_secret_value()
    if apikey == "unset":
        print("No API key present; using a fixed, predetermined value for demonstration purposes")
        return {
            "description": "thunderstorms",
            "temperature": 25.3,
            "humidity": 94
        }

    try:
        # API request to fetch weather data
        weather_url = f"https://api.openweathermap.org/data/2.5/weather?q={location}&appid={apikey}&units=metric"
        weather_data = requests.get(weather_url)
        data = weather_data.json()
        # Extracting relevant weather details
        weather_description = data["weather"][0]["description"]
        temperature = data["main"]["temp"]
        humidity = data["main"]["humidity"]

        # Returning weather details
        return {
            "description": weather_description,
            "temperature": temperature,
            "humidity": humidity
        }
    except Exception as e:
        print(f"Error fetching weather data: {e}")
        return {
            "description": "none",
            "temperature": "none",
            "humidity": "none"
        }

print("=== Weather Tools Initialized ===")
print("  Tool 1: 'get_current_weather'")

## Step 5: Build the Route and Solve Agent

Now we'll build the Route and Solve agent. This agent consists of:
1. A **router node** that analyzes the query and determines which category of tools to use
2. Multiple **subagent nodes**, each specialized for a specific tool category
3. A **loop mechanism** that allows the router to receive feedback and reroute if needed

### Step 5.1: Define the agent state

In [ ]:
from typing import Annotated, TypedDict, Literal
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]
    current_subagent: str

print("State schema defined - Tracks messages and routing decisions for graph flow.")

### Step 5.2: Create subagent graphs

We create separate function calling agents for each tool category using the `create_agent` utility. Each subagent only sees tools relevant to its domain. The `create_agent` function encapsulates the manual graph building (nodes, edges, routing logic), providing a cleaner and more maintainable approach.

In [ ]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage

finance_tools = [get_stock_price]
finance_agent = create_agent(
    model=llm,
    tools=finance_tools,
)

weather_tools = [get_current_weather]
weather_agent = create_agent(
    model=llm,
    tools=weather_tools,
)

print("\n=== Subagents created successfully ===")
print("These subagents will be wrapped as callable tools for the router.")

### Step 5.3: Create router tools

Before creating the router node, we need to define wrapper functions that represent each subagent as a callable tool. These tools will be used by the router's tool-calling mechanism to make routing decisions.

In [ ]:
from langchain_core.messages import HumanMessage
from langchain.tools import tool

@tool
def finance_agent_tool(query: str) -> str:
    """
    Execute financial queries including stock prices, market data, and financial analysis.
    
    This tool executes a specialized finance agent that has access to stock price tools.
    
    Examples of queries this tool handles:
    - "What were the IBM stock prices on 2025-09-05?"
    - "Get me Microsoft stock data for today"
    - "Analyze Apple's recent stock performance"
    
    Use this tool when the user asks about:
    - Stock prices or historical data
    - Financial metrics and market information
    - Investment-related queries
    """
    # Execute the finance agent with the user query
    state = {"messages": [HumanMessage(query)], "current_subagent": ""}
    result = finance_agent.invoke(state)
    # Extract and return the final response
    return result["messages"][-1].content

@tool
def weather_agent_tool(query: str) -> str:
    """
    Execute weather queries including current conditions and forecasts.
    
    This tool executes a specialized weather agent that has access to weather tools.
    
    Examples of queries this tool handles:
    - "What is the weather in San Francisco?"
    - "Get me the current weather in London"
    - "What's the forecast for New York?"
    
    Use this tool when the user asks about:
    - Current weather conditions
    - Weather forecasts
    - Climate and atmospheric information
    """
    
    state = {"messages": [HumanMessage(query)], "current_subagent": ""}
    result = weather_agent.invoke(state)
    
    return result["messages"][-1].content

# Create subagent tools list for LLM tool calling
subagent_tools = [finance_agent_tool, weather_agent_tool]

print("Subagent tools defined as callable functions:")
print("  Tool 1: 'finance_agent_tool' - Executes finance agent with stock price queries")
print("  Tool 2: 'weather_agent_tool' - Executes weather agent with weather queries")

def router_node(state: State) -> State:
    """
    Router node that uses tool calling to invoke the appropriate subagent.
    
    The LLM analyzes the query and decides which tool (subagent) to invoke based on:
    - Tool descriptions and examples
    - The user query content
    - The LLM's understanding of query intent
    
    Unlike traditional routing, this approach directly calls the subagent tool which
    executes the subagent's logic internally, making the architecture more modular
    and extensible.
    
    Returns the state with the subagent's response in messages.
    """
    messages = state["messages"]
    user_query = messages[-1].content if messages else ""
    
    
    llm_with_tools = llm.bind_tools(subagent_tools)
    
    
    response = llm_with_tools.invoke(messages)
    
    if response.tool_calls:
        tool_call = response.tool_calls[0]
        tool_name = tool_call["name"]
        tool_input = tool_call["args"].get("query", user_query)
        
        # Execute the appropriate subagent tool by accessing the underlying function
        # @tool decorator wraps functions in StructuredTool objects, so we have to access the original function
        if tool_name == "finance_agent_tool":
            result = finance_agent_tool.invoke({"query": tool_input})
        elif tool_name == "weather_agent_tool":
            result = weather_agent_tool.invoke({"query": tool_input})
        else:
            result = "Unknown tool requested."
        
        from langchain_core.messages import AIMessage, ToolMessage
        messages = list(messages)
        messages.append(response)
        messages.append(ToolMessage(content=result, tool_call_id=tool_call["id"]))
        
        print(f"[Router]: Query '{user_query[:50]}...' -> Tool called: {tool_name}")
    else:
        
        from langchain_core.messages import AIMessage
        messages = list(messages)
        messages.append(response)
        print(f"[Router]: Query '{user_query[:50]}...' -> No tool needed, using LLM knowledge")
    
    return State(messages=messages, current_subagent="completed")

print("Router node defined - Uses tool calling to invoke subagent tools directly")

### Step 5.4: Create ToolNode for automatic tool invocation

Create a ToolNode that automatically handles subagent tool invocation.

In [ ]:
from langgraph.prebuilt import ToolNode
from langchain_core.messages import AIMessage
from langgraph.graph import END

# Create a ToolNode that automatically handles tool invocation
# ToolNode dispatches based on tool_calls in the LLM response
tool_node = ToolNode(tools=subagent_tools)

print("ToolNode created - Automatically handles subagent tool invocation")

def end_node(state: State) -> State:
    """
    End node that simply returns the state with the final conversation.
    The router has already handled tool execution and added responses to messages.
    """
    return state

print("End node defined - Completes the workflow")

def route_tools(state: State) -> str:
    """
    Route to tools if the last message contains tool calls, otherwise go to end_node.
    """
    messages = state["messages"]
    last_message = messages[-1]
    
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    else:
        return END

print("Route tools function defined - Determines conditional routing logic")

### Step 5.5: Build the graph with ToolNode

Build the graph using the router node, tool node, and conditional routing.

In [ ]:
from langgraph.graph import StateGraph, START, END

# Create the graph
graph_builder = StateGraph(State)

# Add nodes - router, tool node, and end node
graph_builder.add_node("router", router_node)
graph_builder.add_node("tools", tool_node)
graph_builder.add_node("end_node", end_node)

print("Step 1: Nodes added to graph")
print("  - router: Invokes LLM with subagent tools")
print("  - tools: ToolNode handles automatic tool invocation (no manual if/elif)")
print("  - end_node: Returns final state")

# Add edges
# Start -> Router (always start with routing)
graph_builder.add_edge(START, "router")

print("Step 2: Initial edge configured - START -> router")

# Router -> conditional routing to tools or end_node
graph_builder.add_conditional_edges(
    "router",
    route_tools,
    {
        "tools": "tools",
        END: "end_node",
    },
)

print("Step 3: Conditional edges configured")
print("  - router -> tools (if tool calls present)")
print("  - router -> end_node (if no tool calls)")

# Tools -> Back to router (loop for multi-step tool use)
graph_builder.add_edge("tools", "router")

print("Step 4: Tool loop edge configured - tools -> router")

# End node to END
graph_builder.add_edge("end_node", END)

print("Step 5: Final edge configured - end_node -> END")

# Compile the graph
graph = graph_builder.compile()

print("\nStep 6: Route and Solve Agent graph compiled successfully!")
print("\n=== Graph Structure ===")
print("✓ Router node: Invokes LLM with tool binding")
print("✓ ToolNode: Automatically dispatches to subagent tools (no manual if/elif)")
print("✓ Subagent tools: finance_agent_tool and weather_agent_tool")
print("✓ Each subagent: Executes internal agent graph independently")
print("✓ Conditional routing: Based on presence of tool_calls")
print("✓ Loop: Allows multi-step tool invocation for complex queries")

## Step 6: Using the Route and Solve Agent

Now let's test the agent with various queries that will be routed to different subagents.

In [ ]:
from langchain_core.messages import HumanMessage

def route_and_solve_agent(graph, user_input: str):
    """
    Run the Route and Solve agent with a user query.
    
    The agent will:
    1. Route the query to the appropriate subagent
    2. Execute the subagent's tools as needed
    3. Return the final response
    """
    user_message = HumanMessage(user_input)
    print(f"\n{'='*60}")
    print(user_message.pretty_repr())
    print(f"{'='*60}\n")
    
    input_state = {"messages": [user_message], "current_subagent": ""}
    
    for event in graph.stream(input_state):
        for node_name, node_state in event.items():
            if "messages" in node_state and node_state["messages"]:
                print(f"[{node_name}] {node_state['messages'][-1].pretty_repr()}")
                print()

### Test 1: Finance Query

Test a query that should be routed to the finance subagent. The query should include both a ticker symbol and a date in YYYY-MM-DD format.

In [ ]:
route_and_solve_agent(graph, "What were the IBM stock prices on 2025-09-05?")

### Test 2: Weather Query

Test a query that should be routed to the weather subagent.

In [ ]:
route_and_solve_agent(graph, "What is the weather in San Francisco?")

### Test 3: General Knowledge Query

Test a query that doesn't require any specific tool category.

In [ ]:
route_and_solve_agent(graph, "What is the capital of France?")

## Step 7: Simplified Implementation with create_agent

The Route and Solve pattern shown above can also be implemented more concisely using LangChain's `create_agent` utility, just like the Function_Calling_Agent recipe. This is the simplest way to build a routing agent without manually constructing the graph.

Instead of manually building nodes, edges, and routing logic, you can let `create_agent` handle all the complexity internally.

In [ ]:
# Option 1: Using create_agent for maximum simplicity
# This automatically handles graph building, tool routing, and ToolNode integration
router_agent = create_agent(
    model=llm,
    tools=subagent_tools,
)

print("=" * 60)
print("SIMPLIFIED ROUTE AND SOLVE AGENT")
print("=" * 60)
print("\nRouter agent created using create_agent utility")
print("This automatically handles:")
print("  ✓ LLM binding with subagent tools")
print("  ✓ Tool node creation and management")
print("  ✓ Conditional routing logic")
print("  ✓ State management and message handling")

def simplified_route_and_solve(router_agent, user_input: str):
    """
    Run the simplified Route and Solve agent using create_agent.
    
    This demonstrates the most concise way to implement routing.
    """
    user_message = HumanMessage(user_input)
    print(f"\n{'='*60}")
    print(user_message.pretty_repr())
    print(f"{'='*60}\n")
    
    input_state = {"messages": [user_message]}
    
    for event in router_agent.stream(input_state):
        for node_name, node_state in event.items():
            if "messages" in node_state and node_state["messages"]:
                print(f"[{node_name}] {node_state['messages'][-1].pretty_repr()}")
                print()

# Test the simplified agent
print("\n" + "=" * 60)
print("TESTING SIMPLIFIED AGENT")
print("=" * 60)

simplified_route_and_solve(router_agent, "What were the IBM stock prices on 2025-09-05?")
simplified_route_and_solve(router_agent, "What is the weather in San Francisco?")
simplified_route_and_solve(router_agent, "What is the capital of France?")

### Future enhancements to this architecture:

**Extensibility:**
- **Add new subagents**: Simply define a new tool function that wraps a new subagent - no graph modifications needed
- **Multi-domain queries**: Enable the router to call multiple subagent tools sequentially for complex queries
- **Tool composition**: Allow subagents to call tools from other domains through shared tool registries

**Advanced routing:**
- **Confidence scoring**: Have the router evaluate confidence levels before invoking tools
- **Feedback loops**: Allow subagents to report back to the router with partial results or fallback options
- **Dynamic tool discovery**: Automatically register new subagent tools at runtime without code modifications
- **Query disambiguation**: When a query could match multiple tools, use LLM to clarify intent

**Optimization:**
- **Tool caching**: Cache subagent results for identical queries
- **Parallel execution**: Invoke multiple subagent tools concurrently for queries requiring multiple domains
- **Cost optimization**: Route simpler queries to lighter models while reserving heavier models for complex tasks
